## Content


- **Feature Exploration**
    - Create new features


- **Fetching data using pandas**
    - Quering from dataframe - Masking, Filtering, `&` and `|`

- **Apply**
- **Grouping**
    - Split, Apply, Combine
    - `groupby()`
- **Group based Aggregates**

- **Group based Filtering**


#### Loading our IMDB data

In [ ]:
import pandas as pd
import numpy as np
!gdown 1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd
!gdown 1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm
movies = pd.read_csv('movies.csv', index_col=0)
directors = pd.read_csv('directors.csv',index_col=0)
data = movies.merge(directors, how='left', left_on='director_id',right_on='id')  
data.drop(['director_id','id_y'],axis=1,inplace=True)

Downloading...
From: https://drive.google.com/uc?id=1s2TkjSpzNc4SyxqRrQleZyDIHlc7bxnd
To: /content/movies.csv
100% 112k/112k [00:00<00:00, 81.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ws-_s1fHZ9nHfGLVUQurbHDvStePlEJm
To: /content/directors.csv
100% 65.4k/65.4k [00:00<00:00, 62.7MB/s]


## Feature Exploration


Lets explore all the features in the merged dataset



In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1465 entries, 0 to 1464
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_x           1465 non-null   int64  
 1   budget         1465 non-null   int64  
 2   popularity     1465 non-null   int64  
 3   revenue        1465 non-null   int64  
 4   title          1465 non-null   object 
 5   vote_average   1465 non-null   float64
 6   vote_count     1465 non-null   int64  
 7   year           1465 non-null   int64  
 8   month          1465 non-null   object 
 9   day            1465 non-null   object 
 10  director_name  1465 non-null   object 
 11  gender         1341 non-null   object 
dtypes: float64(1), int64(6), object(5)
memory usage: 148.8+ KB


Looks like only `gender` column has missing values (will come later)

#### How can we describe these features to know more about their range of values?

In [ ]:
data.describe()

,id_x,budget,popularity,revenue,vote_average,vote_count,year
count,1465.000000,1.465000e+03,1465.000000,1.465000e+03,1465.000000,1465.000000,1465.000000
mean,45225.191126,4.802295e+07,30.855973,1.432539e+08,6.368191,1146.396587,2002.615017
std,1189.096396,4.935541e+07,34.845214,2.064918e+08,0.818033,1578.077438,8.680141
min,43597.000000,0.000000e+00,0.000000,0.000000e+00,3.000000,1.000000,1976.000000
25%,44236.000000,1.400000e+07,11.000000,1.738013e+07,5.900000,216.000000,1998.000000
50%,45022.000000,3.300000e+07,23.000000,7.578164e+07,6.400000,571.000000,2004.000000
75%,45990.000000,6.600000e+07,41.000000,1.792469e+08,6.900000,1387.000000,2009.000000
max,48395.000000,3.800000e+08,724.000000,2.787965e+09,8.300000,13752.000000,2016.000000


This gives us all **statistical properties** of the columns

If you notice, some columns such as "title", "month" are missing

How are these **missing columns different?**

They are of **object dtype**

#### Then how can we include object type in `df.describe()`?

In [ ]:
data.describe(include=object)

,title,month,day,director_name,gender
count,1465,1465,1465,1465,1341
unique,1465,12,7,199,2
top,Avatar,Dec,Friday,Steven Spielberg,Male
freq,1,193,654,26,1309


If you notice,

- The range of values in the `revenue` and `budget` seem to be very high
- Generally budget and revenue for Hollywood movies is in million dollars


<!-- So it will be better to change the values into `million dollars USD`  -->

#### How can we change the values of `revenue` and `budget` into million dollars USD?


In [ ]:
data['revenue'] = (data['revenue']/1000000).round(2)
data


,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237000000,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
1,43598,300000000,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
2,43599,245000000,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,Male
3,43600,250000000,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
4,43602,258000000,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0,3,0.32,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,Male
1461,48370,27000,19,3.15,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,Male
1462,48375,0,7,0.00,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,Male
1463,48376,0,3,0.00,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,Male


Similarly, we can do it for 'budget' as well

In [ ]:
data['budget']=(data['budget']/1000000).round(2)
data.head()

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237.0,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
1,43598,300.0,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
2,43599,245.0,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,Male
3,43600,250.0,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
4,43602,258.0,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,Male


## Fetching queries from dataframe



Lets say we are interested in fetching all **highly rated movies**
- say movies with **ratings > 7**

#### How can we get movies with ratings > 7?

We can use the concept of `masking`

<!-- -  Have you ever come across or seen SQL?
- For now, just know that **SQL is used to fetch data from databases**
- We can use basic **Pandas operations** in a similar way to **fetch desired data from loaded data** -->


Lets first create a mask to filter such movies 
- In SQL: ```SELECT * FROM movies WHERE vote_average > 7```
- In pandas:

In [ ]:
data['vote_average'] > 7

0        True
1       False
2       False
3        True
4       False
        ...  
1460     True
1461     True
1462    False
1463    False
1464    False
Name: vote_average, Length: 1465, dtype: bool

But we still don't know the row values ... Only that which row satisfied the condtion

#### How do we get the row values from this mask?

In [ ]:
data.loc[data['vote_average'] > 7]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237.00,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
3,43600,250.00,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
14,43616,250.00,120,956.02,The Hobbit: The Battle of the Five Armies,7.1,4760,2014,Dec,Wednesday,Peter Jackson,Male
16,43619,250.00,94,958.40,The Hobbit: The Desolation of Smaug,7.6,4524,2013,Dec,Wednesday,Peter Jackson,Male
19,43622,200.00,100,1845.03,Titanic,7.5,7562,1997,Nov,Tuesday,James Cameron,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
1456,48321,0.01,20,7.00,Eraserhead,7.5,485,1977,Mar,Saturday,David Lynch,Male
1457,48323,0.00,5,0.00,The Mighty,7.1,51,1998,Oct,Friday,Peter Chelsom,Male
1458,48335,0.06,27,3.22,Pi,7.1,586,1998,Jul,Friday,Darren Aronofsky,Male
1460,48363,0.00,3,0.32,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,Male



You can also perform the filtering without even using `loc`




In [ ]:
data[data['vote_average'] > 7]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237.00,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,Male
3,43600,250.00,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,Male
14,43616,250.00,120,956.02,The Hobbit: The Battle of the Five Armies,7.1,4760,2014,Dec,Wednesday,Peter Jackson,Male
16,43619,250.00,94,958.40,The Hobbit: The Desolation of Smaug,7.6,4524,2013,Dec,Wednesday,Peter Jackson,Male
19,43622,200.00,100,1845.03,Titanic,7.5,7562,1997,Nov,Tuesday,James Cameron,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
1456,48321,0.01,20,7.00,Eraserhead,7.5,485,1977,Mar,Saturday,David Lynch,Male
1457,48323,0.00,5,0.00,The Mighty,7.1,51,1998,Oct,Friday,Peter Chelsom,Male
1458,48335,0.06,27,3.22,Pi,7.1,586,1998,Jul,Friday,Darren Aronofsky,Male
1460,48363,0.00,3,0.32,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,Male


But this is not recommended. Why ?
  - It can create a confusion between implicit/explicit indexing used as discussed before
  - `loc` is also much faster


#### Now, how can we return a subset of columns, say, only `title` and `director_name`?

In [ ]:
data.loc[data['vote_average'] > 7, ['title','director_name']]

,title,director_name
0,Avatar,James Cameron
3,The Dark Knight Rises,Christopher Nolan
14,The Hobbit: The Battle of the Five Armies,Peter Jackson
16,The Hobbit: The Desolation of Smaug,Peter Jackson
19,Titanic,James Cameron
...,...,...
1456,Eraserhead,David Lynch
1457,The Mighty,Peter Chelsom
1458,Pi,Darren Aronofsky
1460,The Last Waltz,Martin Scorsese


So far we saw only single condition for filtering 

#### What if we want to filter highly rated movies released after 2014?

Notice that two conditions are involved here
1. Movies need to be highly rated i.e.. > 7
2. They should be 2015 and onwards


We can **use AND operator b/w multiple conditions**

In [ ]:
data.loc[(data['vote_average'] > 7) & (data['year'] >= 2015)].head()

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
30,43641,190.0,102,1506.25,Furious 7,7.3,4176,2015,Apr,Wednesday,James Wan,Male
78,43724,150.0,434,378.86,Mad Max: Fury Road,7.2,9427,2015,May,Wednesday,George Miller,Male
106,43773,135.0,100,532.95,The Revenant,7.3,6396,2015,Dec,Friday,Alejandro González Iñárritu,Male
162,43867,108.0,167,630.16,The Martian,7.6,7268,2015,Sep,Wednesday,Ridley Scott,Male
312,44128,75.0,48,108.15,The Man from U.N.C.L.E.,7.1,2265,2015,Aug,Thursday,Guy Ritchie,Male


Recall how we apply **mutliple conditions in numpy ?**

Use **elementwise operator `&` or `|`**

Note:
-  **we cannot use `and` or `or`** with dataframe 
<!-- as a  dataframe has **multiple values** -->
- **for multiple conditions**, we need to put each **separate condition within parenthesis `()`**

#### Similarly how can we find movies released on either Friday or Sunday?

In [ ]:
data.loc[(data['day'] == 'Friday') | (data['day'] == 'Saturday')].head()

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
1,43598,300.0,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,Male
12,43614,380.0,135,1045.71,Pirates of the Caribbean: On Stranger Tides,6.4,4948,2011,May,Saturday,Rob Marshall,Male
22,43627,200.0,35,783.77,Spider-Man 2,6.7,4321,2004,Jun,Friday,Sam Raimi,Male
25,43632,150.0,21,836.30,Transformers: Revenge of the Fallen,6.0,3138,2009,Jun,Friday,Michael Bay,Male
40,43656,200.0,45,769.65,2012,5.6,4903,2009,Oct,Saturday,Roland Emmerich,Male


Thus we can do complex queries using both ```&``` and ```|``` operators 

Now let's try to answer few more Questions from this data

#### How will you find Top 5 most popular movies?

We can simply sort our data based on values of column 'popularity'

In [ ]:
data.sort_values(['popularity'],ascending=False).head(5)

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
58,43692,165.0,724,675.12,Interstellar,8.1,10867,2014,Nov,Wednesday,Christopher Nolan,Male
78,43724,150.0,434,378.86,Mad Max: Fury Road,7.2,9427,2015,May,Wednesday,George Miller,Male
119,43796,140.0,271,655.01,Pirates of the Caribbean: The Curse of the Bla...,7.5,6985,2003,Jul,Wednesday,Gore Verbinski,Male
120,43797,125.0,206,752.10,The Hunger Games: Mockingjay - Part 1,6.6,5584,2014,Nov,Tuesday,Francis Lawrence,Male
45,43662,185.0,187,1004.56,The Dark Knight,8.2,12002,2008,Jul,Wednesday,Christopher Nolan,Male


On applying this to a string column, it sorts the dataframe ***lexicographically**

In [ ]:
data.sort_values(['title'],ascending=False).head(5)

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
436,44364,60.0,36,71.07,xXx: State of the Union,4.7,549,2005,Apr,Wednesday,Lee Tamahori,Male
330,44165,70.0,46,277.45,xXx,5.8,1424,2002,Aug,Friday,Rob Cohen,Male
994,45681,15.0,21,2.86,eXistenZ,6.7,475,1999,Apr,Wednesday,David Cronenberg,Male
547,44594,50.0,37,55.97,Zoolander 2,4.7,797,2016,Feb,Saturday,Ben Stiller,Male
850,45313,28.0,38,60.78,Zoolander,6.1,1337,2001,Sep,Friday,Ben Stiller,Male


#### Now, how will get list of movies directed by a particular director, say, 'Christopher Nolan'?

In [ ]:
data.loc[data['director_name'] == 'Christopher Nolan',['title']]

,title
3,The Dark Knight Rises
45,The Dark Knight
58,Interstellar
59,Inception
74,Batman Begins
565,Insomnia
641,The Prestige
1341,Memento


Note: 
- The string indicating "Christopher Nolan" could have been something else as well.  
- The better way is to use string methods, we will discuss this later

## Apply

Now suppose we want to convert our `Gender` column data to numerical format

Basically, 
- 0 for Male
- 1 for Female

#### How can we encode the column?

Let's first write a function to do it for a single value




In [ ]:
def encode(data):
  if data == "Male":
    return 0
  else:
    return 1 

#### Now how can we apply this function to the whole column?


In [ ]:
data['gender'] = data['gender'].apply(encode)
data

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender
0,43597,237.00,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0
1,43598,300.00,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0
2,43599,245.00,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0
3,43600,250.00,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0
4,43602,258.00,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0.00,3,0.32,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0
1461,48370,0.03,19,3.15,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0
1462,48375,0.00,7,0.00,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0
1463,48376,0.00,3,0.00,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0


Notice how this is similar to using vectorization in Numpy

We thus can use `apply` to use a function throughout a column

Can we **use apply on multiple columns**?

Say,
#### How to find sum of revenue and budget per movie?

In [ ]:
data[['revenue', 'budget']].apply(np.sum)

revenue    209867.04
budget      70353.62
dtype: float64

We can pass **multiple cols by packing them** within `[]`

But there's a mistake here. We wanted our results per movie (per row)

But, we are getting the sum of the columns

#### How can we use apply to work on individual rows?

In [ ]:
data[['revenue', 'budget']].apply(np.sum, axis=1)

0       3024.97
1       1261.00
2       1125.67
3       1334.94
4       1148.87
         ...   
1460       0.32
1461       3.18
1462       0.00
1463       0.00
1464       2.26
Length: 1465, dtype: float64

Every row of `revenue` was added to same row of `budget`

#### What does this `axis` mean in apply ?
  - If **axis = 0**, it will apply to **each column**, if **axis = 1**, **each row**
  - By default axis = 0

=> `apply()` can be applied on any dataframe along any particular axis


#### Similarly, how can I find profit per movie (revenue-budget)?

In [ ]:
def prof(x): # We define a function to calculate profit
  return x['revenue']-x['budget']
data['profit'] = data[['revenue', 'budget']].apply(prof, axis = 1)
data

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit
0,43597,237.00,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550.97
1,43598,300.00,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661.00
2,43599,245.00,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635.67
3,43600,250.00,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834.94
4,43602,258.00,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,48363,0.00,3,0.32,The Last Waltz,7.9,64,1978,May,Monday,Martin Scorsese,0,0.32
1461,48370,0.03,19,3.15,Clerks,7.4,755,1994,Sep,Tuesday,Kevin Smith,0,3.12
1462,48375,0.00,7,0.00,Rampage,6.0,131,2009,Aug,Friday,Uwe Boll,0,0.00
1463,48376,0.00,3,0.00,Slacker,6.4,77,1990,Jul,Friday,Richard Linklater,0,0.00


Thus, we can access the columns by their names inside the functions too using apply

## Grouping



#### How can we know the number of movies released by a particular director, say, Christopher Nolan?

In [ ]:
data.loc[data['director_name'] == 'Christopher Nolan',['title']].count()

title    8
dtype: int64

#### What if we have to do find number of movies of each director? 

We have value_counts() for this

In [ ]:
data["director_name"].value_counts()

Steven Spielberg      26
Martin Scorsese       19
Clint Eastwood        19
Woody Allen           18
Ridley Scott          16
                      ..
Tim Hill               5
Jonathan Liebesman     5
Roman Polanski         5
Larry Charles          5
Nicole Holofcener      5
Name: director_name, Length: 199, dtype: int64

How does this exactly work? 

We can assume pandas must have **grouped the rows internally** to find the count

But what if we need to find some **other metric** besides count?

For example, **average popularity** of each director, or **max rating** among all movies by a director?

#### How can you find the average popularity of each director?

We will have to some group our rows director wise.


#### What is Grouping ?

Simply it could be understood through the terms - Split, apply, combine

<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781491912126/files/assets/pyds_03in01.png" height = 350/>

1. **Split**: **Breaking up and grouping** a DataFrame depending on the value of the specified key.

2. **Apply**: Computing **some functio**n, usually an **aggregate, transformation, or filtering**, within the individual groups.

3. **Combine**: **Merge the results** of these operations into an output array.

Note: 

All these steps are to understand the topic

## Group based Aggregates


#### Now, how can we group our data director-wise?


In [ ]:
data.groupby('director_name')

Notice,
- It's a **DataFrameGroupBy type object**

- **NOT a DataFrame** type object

#### What is `groupby('director_name')` doing?

**Grouping all rows** in which **director_name** value is **same**

But it's returning an object, we would want to get information out of this object.

Let's look at few attributes of the same.





#### How can we know the number of groups our data is divided into?

In [ ]:
data.groupby('director_name').ngroups

199

<!-- Access group and corresponding row keys -->

Based on this grouping, how can we find which keys belong to which group?

In [ ]:
data.groupby('director_name').groups

{'Adam McKay': [176, 323, 366, 505, 839, 916], 'Adam Shankman': [265, 300, 350, 404, 458, 843, 999, 1231], 'Alejandro González Iñárritu': [106, 749, 1015, 1034, 1077, 1405], 'Alex Proyas': [95, 159, 514, 671, 873], 'Alexander Payne': [793, 1006, 1101, 1211, 1281], 'Andrew Adamson': [11, 43, 328, 501, 947], 'Andrew Niccol': [533, 603, 701, 722, 1439], 'Andrzej Bartkowiak': [349, 549, 754, 911, 924], 'Andy Fickman': [517, 681, 909, 926, 973, 1023], 'Andy Tennant': [314, 320, 464, 593, 676, 885], 'Ang Lee': [99, 134, 748, 840, 1089, 1110, 1132, 1184], 'Anne Fletcher': [610, 650, 736, 789, 1206], 'Antoine Fuqua': [310, 338, 424, 467, 576, 808, 818, 1105], 'Atom Egoyan': [946, 1128, 1164, 1194, 1347, 1416], 'Barry Levinson': [313, 319, 471, 594, 878, 898, 1013, 1037, 1082, 1143, 1185, 1345, 1378], 'Barry Sonnenfeld': [13, 48, 90, 205, 591, 778, 783], 'Ben Stiller': [209, 212, 547, 562, 850], 'Bill Condon': [102, 307, 902, 1233, 1381], 'Bobby Farrelly': [352, 356, 481, 498, 624, 630, 654, 80

<!-- Access data for specific group -->

#### Now what if we want to extract data of a particular group from this list?

In [ ]:
data.groupby('director_name').get_group('Alexander Payne')

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit
793,45163,30.0,19,105.83,About Schmidt,6.7,362,2002,Dec,Friday,Alexander Payne,1,75.83
1006,45699,20.0,40,177.24,The Descendants,6.7,934,2011,Sep,Friday,Alexander Payne,1,157.24
1101,46004,16.0,23,109.50,Sideways,6.9,478,2004,Oct,Friday,Alexander Payne,1,93.50
1211,46446,12.0,29,17.65,Nebraska,7.4,636,2013,Sep,Saturday,Alexander Payne,1,5.65
1281,46813,0.0,13,0.00,Election,6.7,270,1999,Apr,Friday,Alexander Payne,1,0.00


Great! We are able to extract the data from our DataFrameGroupBy object

But can we extend this to finding an aggregate metric of the data?
#### How can we find average popularity of each director?

In [ ]:
data.groupby('director_name').mean()

,id_x,budget,popularity,revenue,vote_average,vote_count,year,gender,profit
director_name,,,,,,,,,
Adam McKay,44586.000000,56.916667,30.333333,143.180000,6.466667,1326.500000,2009.333333,0.0,86.263333
Adam Shankman,44821.250000,48.375000,23.125000,109.196250,6.037500,623.875000,2005.375000,0.0,60.821250
Alejandro González Iñárritu,45660.500000,33.333333,47.000000,146.331667,7.233333,2286.000000,2008.000000,0.0,112.998333
Alex Proyas,44477.000000,70.400000,53.200000,154.912000,6.480000,1667.400000,2004.200000,0.0,84.512000
Alexander Payne,46025.000000,15.600000,24.800000,82.044000,6.880000,536.000000,2005.800000,1.0,66.444000
...,...,...,...,...,...,...,...,...,...
Wes Craven,45503.300000,23.380000,22.300000,76.478000,5.950000,630.500000,1999.700000,0.0,53.098000
Wolfgang Petersen,44511.285714,90.142857,35.857143,230.717143,6.571429,986.714286,1995.285714,0.0,140.574286
Woody Allen,46083.777778,11.777778,17.722222,34.495000,6.672222,504.111111,2001.611111,0.0,22.717222


This does give us the max value of the data, but for **all the features**

How can we **specify a single feature**, such as **popularity**, in this case?

In [ ]:
data.groupby('director_name')['popularity'].mean()

director_name
Adam McKay                     30.333333
Adam Shankman                  23.125000
Alejandro González Iñárritu    47.000000
Alex Proyas                    53.200000
Alexander Payne                24.800000
                                 ...    
Wes Craven                     22.300000
Wolfgang Petersen              35.857143
Woody Allen                    17.722222
Zack Snyder                    71.857143
Zhang Yimou                    12.000000
Name: popularity, Length: 199, dtype: float64

Now say we want to know two aggregations for any feature.

For e.g., the very first year and the latest year a director released a movie

This is basically the `min` and `max` of `year` column, grouped by director

#### How can we find multiple aggregations of any feature?

In [ ]:
data.groupby(['director_name'])["year"].aggregate(['min', 'max'])


,min,max
director_name,,
Adam McKay,2004,2015
Adam Shankman,2001,2012
Alejandro González Iñárritu,2000,2015
Alex Proyas,1994,2016
Alexander Payne,1999,2013
...,...,...
Wes Craven,1984,2011
Wolfgang Petersen,1981,2006
Woody Allen,1977,2013


## Group based Filtering



#### How we find details of the movies by high budget directors?


Lets assume, 
- high budget director -> any director with **atleast one movie with budget >100M**  

  
<!-- Notice, the question is **not asking us to give the name of the directors who have directed high budget movies** 

Lets first quickly see, how we would  have solved that 
To begin with, 

#### How can we find the names of high budget directors? -->

We can get the highest budget movie data of every director

<!-- We can,

1. `group` the data by director
2. use `max` of the budget as aggregator -->

In [ ]:
data_dir_budget = data.groupby("director_name")["budget"].max().reset_index()
data_dir_budget.head()

,director_name,budget
0,Adam McKay,100.0
1,Adam Shankman,80.0
2,Alejandro González Iñárritu,135.0
3,Alex Proyas,140.0
4,Alexander Payne,30.0


#### How can we **filter** out the director names with **max budget >100M**?

In [ ]:
names = data_dir_budget.loc[data_dir_budget["budget"] >= 100, "director_name"]

#### Finally, how can we filter out the details of the movies by these directors?

In [ ]:
data.loc[data['director_name'].isin(names)]

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit
0,43597,237.00,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550.97
1,43598,300.00,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661.00
2,43599,245.00,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635.67
3,43600,250.00,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834.94
4,43602,258.00,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450,48267,0.40,33,100.00,Mad Max,6.6,1213,1979,Apr,Thursday,George Miller,0,99.60
1451,48268,0.20,13,4.51,Swingers,6.8,253,1996,Oct,Friday,Doug Liman,0,4.31
1452,48274,0.00,5,2.61,Three,6.3,31,2010,Dec,Thursday,Tom Tykwer,0,2.61
1458,48335,0.06,27,3.22,Pi,7.1,586,1998,Jul,Friday,Darren Aronofsky,0,3.16


Recall `isin()` from last lecture

#### Can we do filtering of groups in a single go?

**YES**

In [ ]:
data.groupby('director_name').filter(lambda x: x["budget"].max() >= 100)

,id_x,budget,popularity,revenue,title,vote_average,vote_count,year,month,day,director_name,gender,profit
0,43597,237.00,150,2787.97,Avatar,7.2,11800,2009,Dec,Thursday,James Cameron,0,2550.97
1,43598,300.00,139,961.00,Pirates of the Caribbean: At World's End,6.9,4500,2007,May,Saturday,Gore Verbinski,0,661.00
2,43599,245.00,107,880.67,Spectre,6.3,4466,2015,Oct,Monday,Sam Mendes,0,635.67
3,43600,250.00,112,1084.94,The Dark Knight Rises,7.6,9106,2012,Jul,Monday,Christopher Nolan,0,834.94
4,43602,258.00,115,890.87,Spider-Man 3,5.9,3576,2007,May,Tuesday,Sam Raimi,0,632.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450,48267,0.40,33,100.00,Mad Max,6.6,1213,1979,Apr,Thursday,George Miller,0,99.60
1451,48268,0.20,13,4.51,Swingers,6.8,253,1996,Oct,Friday,Doug Liman,0,4.31
1452,48274,0.00,5,2.61,Three,6.3,31,2010,Dec,Thursday,Tom Tykwer,0,2.61
1458,48335,0.06,27,3.22,Pi,7.1,586,1998,Jul,Friday,Darren Aronofsky,0,3.16


Notice what's happening here?
- We first group data by director and then use `groupby().filter` function
- **Groups are filtered if they do not satisfy the boolean criterion** specified by function
- This is called  **Group Based Filtering**



**NOTE**

We are filtering the **groups** here and **not the rows**

==> The result is **not a groupby object** but **regular pandas DataFrame** with the **filtered groups eliminated**